In [12]:
import pandas as pd

import re
import nltk
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import STOPWORDS as gensim_words
import spacy
sp = spacy.load('en_core_web_sm')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import time

In [13]:
nltk_stopwords = set(stopwords.words('english')) \
                    .union(set(stopwords.words('german'))) \
                    .union(set(stopwords.words('french')))
gensim_stopwords = set(gensim_words)
spacy_stopwords = sp.Defaults.stop_words
# https://countwordsfree.com/stopwords
cwf_stopwords = set(line.strip() for line in open('stop_words.txt'))

all_stopwords = list( nltk_stopwords \
                        .union(gensim_stopwords) \
                        .union(spacy_stopwords) \
                        .union(cwf_stopwords) )

In [14]:
df = pd.read_csv('/home/rikz/Documents/Master/Semester2/SDDM/data/data_full.csv', index_col=0)
df.head()

,paper_id,title,list_authors,full_text
0,question0,-,-,How does temperature and humidity affect the t...
1,question1,-,-,Seasonality of transmission
2,question2,-,-,Effectiveness of inter inner travel restriction
3,question3,-,-,Effectiveness of personal protective equipment...
4,question4,-,-,Effectiveness of school distancing


In [15]:
question_num = 2

df_questions = pd.read_csv('/home/rikz/Documents/Master/Semester2/SDDM/data/questions.csv')
df_questions = df_questions.loc[df_questions.question_id == 2]

df_questions.head()

,question_id,full_text
2,2,Effectiveness of inter inner travel restriction


In [16]:
time_before = time.time()

In [17]:
# Peprocess the data
print("Before removing empty papers and duplicates: {} rows.".format(len(df)))
df = df.dropna(subset=['full_text'])
print("Removed empty papers")
df = df.drop_duplicates(subset=['full_text'])
print("Removed duplicates")
print("After removing empty papers and duplicates: {} rows.".format(len(df)))

df.head()

Before removing empty papers and duplicates: 80754 rows.
Removed empty papers
Removed duplicates
After removing empty papers and duplicates: 79570 rows.


,paper_id,title,list_authors,full_text
0,question0,-,-,How does temperature and humidity affect the t...
1,question1,-,-,Seasonality of transmission
2,question2,-,-,Effectiveness of inter inner travel restriction
3,question3,-,-,Effectiveness of personal protective equipment...
4,question4,-,-,Effectiveness of school distancing


In [ ]:
def preprocess(text):
    tokenizer = nltk.tokenize.WhitespaceTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    # Make text lower case
    preprocessed = text.lower()
    # Remove numbers and special characters
    preprocessed = re.sub('[^a-z ]+', '', preprocessed)
    # Remove white spaces
    preprocessed = preprocessed.strip()
    # Lemmatize text
    preprocessed = ''.join(lemmatizer.lemmatize(w)+' ' for w in tokenizer.tokenize(preprocessed))
    
    return preprocessed
    
df['preprocessed'] = df.full_text.apply(preprocess)

In [ ]:
df.head()

In [ ]:
vectorizer = CountVectorizer(stop_words=all_stopwords)
vectorizer.fit_transform(df['preprocessed'])

In [ ]:
time_after = time.time()

In [ ]:
print('Preprocessing time: {} sec'.format(time_after-time_before) )